In [1]:
# coding: utf-8
import os
import sys
from keras.models import Model,load_model
from keras.layers import Input, LSTM, Dense,Activation
from keras.layers.core import Lambda
import numpy as np
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from nltk.translate.bleu_score import sentence_bleu
from keras.callbacks import EarlyStopping

post='stc_weibo_train_post'
response='stc_weibo_train_response'

batch_size=32
sample_per_epochs=2000
max_num_of_samples=100000
epochs=100
latent_dim=128
split_word=" "
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
KTF.set_session(session)


Using TensorFlow backend.


In [2]:
input_texts=[]
target_texts=[]
token_index={}
token_index['\t']=0
token_index['\n']=1

In [3]:
que_len=20
ans_len=10

In [5]:
with open(post, encoding = 'utf8') as f:
    input_lines=f.readlines()
with open(response, encoding = 'utf8')  as f:
    target_lines=f.readlines()

input_texts=[]
target_texts=[]
for i in range(len(input_lines)):
    if len(input_texts)>=max_num_of_samples:
        break
    if 3<len(input_lines[i].replace("\n","").split(split_word))<que_len and 3<len(target_lines[i].replace("\n","").split(split_word))<ans_len-2:
        input_texts.append(input_lines[i].replace("\n","").split(split_word))
        target_texts.append(["\t"]+target_lines[i].replace("\n","").split(split_word)+["\n"])
        for word in input_texts[-1]:
            if word not in token_index:
                token_index[word]=len(token_index)
        for word in target_texts[-1]:
            if word not in token_index:
                token_index[word]=len(token_index)

num_tokens=len(token_index)
num_samples=len(input_texts)
print("Num of samples:",num_samples )
print("Num of unique token:",num_tokens)


Num of samples: 100000
Num of unique token: 94419


In [6]:
encoder_inputs=Input(shape=(None,num_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

decoder_inputs=Input(shape=(None,num_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_tokens) #,activation='softmax'
decoder_weighted=Lambda((lambda x: x/0.7))
decoder_softmax=Activation('softmax')
decoder_outputs=decoder_softmax(decoder_weighted(decoder_dense(decoder_outputs)))
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
cbes=EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 94419)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 94419)  0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 48408576    input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  48408576    input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [ ]:
for epoch in range(epochs):
    for batch in range((num_samples//sample_per_epochs)-1):
        print("EPOCH",epoch,"BATCH",batch+1)fa
        decoder_input_data=np.zeros(
            (sample_per_epochs,ans_len,num_tokens),dtype='float32'
        )
        decoder_target_data=np.zeros(
            (sample_per_epochs,ans_len,num_tokens),dtype='float32'
        )
        encoder_input_data=np.zeros(
            (sample_per_epochs,que_len,num_tokens),dtype='float32'
        )
        for i in range(sample_per_epochs):
            real_index=batch*sample_per_epochs+i
            for t,word in enumerate(input_texts[real_index]):
                encoder_input_data[i,t,token_index[word]]=1.
            for t,word in enumerate(target_texts[real_index]):
                decoder_input_data[i,t,token_index[word]]=1.
                if t>0:
                    decoder_target_data[i,t-1,token_index[word]]=1.
        model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=1,validation_split=0.2,callbacks=[cbes])

EPOCH 0 BATCH 1
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 46s 29ms/step - loss: 6.0169 - acc: 0.0983 - val_loss: 5.4672 - val_acc: 0.1000
EPOCH 0 BATCH 2
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 26ms/step - loss: 5.2583 - acc: 0.1191 - val_loss: 5.2158 - val_acc: 0.1208
EPOCH 0 BATCH 3
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 26ms/step - loss: 5.1309 - acc: 0.1217 - val_loss: 5.0695 - val_acc: 0.1193
EPOCH 0 BATCH 4
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 27ms/step - loss: 5.0592 - acc: 0.1247 - val_loss: 5.0498 - val_acc: 0.1230
EPOCH 0 BATCH 5
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 27ms/step - loss: 5.0335 - acc: 0.1256 - val_loss: 5.0888 - val_acc: 0.1223
EPOCH 0 BA

1600/1600 [==============================] - 42s 26ms/step - loss: 4.7582 - acc: 0.1448 - val_loss: 4.8253 - val_acc: 0.1395
EPOCH 0 BATCH 43
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 26ms/step - loss: 4.7344 - acc: 0.1444 - val_loss: 4.7555 - val_acc: 0.1480
EPOCH 0 BATCH 44
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 26ms/step - loss: 4.7642 - acc: 0.1448 - val_loss: 4.7397 - val_acc: 0.1450
EPOCH 0 BATCH 45
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 26ms/step - loss: 4.7562 - acc: 0.1455 - val_loss: 4.6836 - val_acc: 0.1495
EPOCH 0 BATCH 46
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 42s 26ms/step - loss: 4.7729 - acc: 0.1457 - val_loss: 4.7629 - val_acc: 0.1460
EPOCH 0 BATCH 47
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/

1600/1600 [==============================] - 44s 28ms/step - loss: 4.5742 - acc: 0.1547 - val_loss: 4.6368 - val_acc: 0.1588
EPOCH 1 BATCH 35
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 44s 28ms/step - loss: 4.6164 - acc: 0.1520 - val_loss: 4.6669 - val_acc: 0.1605
EPOCH 1 BATCH 36
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 44s 28ms/step - loss: 4.5968 - acc: 0.1567 - val_loss: 4.6460 - val_acc: 0.1550
EPOCH 1 BATCH 37
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 44s 28ms/step - loss: 4.5931 - acc: 0.1512 - val_loss: 4.6835 - val_acc: 0.1523
EPOCH 1 BATCH 38
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/1600 [==============================] - 44s 28ms/step - loss: 4.5883 - acc: 0.1544 - val_loss: 4.6800 - val_acc: 0.1545
EPOCH 1 BATCH 39
Train on 1600 samples, validate on 400 samples
Epoch 1/1
1600/

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_word_index=dict((i,char) for char,i in token_index.items())


decoder_input_data=np.zeros(
    (sample_per_epochs,ans_len,num_tokens),dtype='float32'
)
decoder_target_data=np.zeros(
    (sample_per_epochs,ans_len,num_tokens),dtype='float32'
)
encoder_input_data=np.zeros(
    (sample_per_epochs,que_len,num_tokens),dtype='float32'
)
for i in range(sample_per_epochs):
    real_index=((num_samples//sample_per_epochs)-1)*sample_per_epochs+i
    for t,word in enumerate(input_texts[real_index]):
        encoder_input_data[i,t,token_index[word]]=1.
    for t,word in enumerate(target_texts[real_index]):
        decoder_input_data[i,t,token_index[word]]=1.
        if t>0:
            decoder_target_data[i,t-1,token_index[word]]=1.


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    decoded_sequence=[]
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_word_index[sampled_token_index]
        decoded_sentence += sampled_char
        decoded_sequence.append(sampled_char)
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence,decoded_sequence

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence,seq = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('BLEU:',sentence_bleu(target_texts, seq))

